# <center> Data Cleaning and Exploration </center>

This notebook is primarily used to analyze the data at a basic level and identify any discrepencies that may affect the future parts of this project.

This consists of:
- Understanding the data (which contains at least 7000 realistic climate records covering temperature, rainfall and AQI in major Indian cities)
- Identifying missing values and duplicates, if any
- Conversion of column data types, if needed
- Flagging any invalid or inconsistent values
- Checking other problems with the data

In [3]:
import pandas as pd
# import data
data = pd.read_csv("data.csv")

# Understanding the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7310 entries, 0 to 7309
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  7310 non-null   object 
 1   City                  7310 non-null   object 
 2   State                 7310 non-null   object 
 3   Temperature_Max (°C)  7310 non-null   float64
 4   Temperature_Min (°C)  7310 non-null   float64
 5   Temperature_Avg (°C)  7310 non-null   float64
 6   Humidity (%)          7310 non-null   float64
 7   Rainfall (mm)         7310 non-null   float64
 8   Wind_Speed (km/h)     7310 non-null   float64
 9   AQI                   7310 non-null   int64  
 10  AQI_Category          7310 non-null   object 
 11  Pressure (hPa)        7310 non-null   float64
 12  Cloud_Cover (%)       7310 non-null   float64
dtypes: float64(8), int64(1), object(4)
memory usage: 742.6+ KB


Based on the summary above, there are no null values in the raw data. We can also see each column data type and potentially convert a few column types to another for better inference.

As seen below, there are also no duplicates.

In [4]:
# Check for duplicates (there should not be any)
data.duplicated().sum()

np.int64(0)

Convert:
- the date column from object to datetime
- the city, state columns from object to string
- the AQI_Category column from object to category

In [5]:
data['Date'] = pd.to_datetime(data['Date'])
data['City'] = data['City'].astype('string')
data['State'] = data['State'].astype('string')
data['AQI_Category'] = data['AQI_Category'].astype('category')

In [6]:
# Check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7310 entries, 0 to 7309
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  7310 non-null   datetime64[ns]
 1   City                  7310 non-null   string        
 2   State                 7310 non-null   string        
 3   Temperature_Max (°C)  7310 non-null   float64       
 4   Temperature_Min (°C)  7310 non-null   float64       
 5   Temperature_Avg (°C)  7310 non-null   float64       
 6   Humidity (%)          7310 non-null   float64       
 7   Rainfall (mm)         7310 non-null   float64       
 8   Wind_Speed (km/h)     7310 non-null   float64       
 9   AQI                   7310 non-null   int64         
 10  AQI_Category          7310 non-null   category      
 11  Pressure (hPa)        7310 non-null   float64       
 12  Cloud_Cover (%)       7310 non-null   float64       
dtypes: category(1), da

In [26]:
# Check if the date column is ordered correctly, and in turn, the rows of the dataset
print(data['Date'].is_monotonic_increasing)

True


### Invalid/inconsistent values

With weather data, it is important to check if features such as humidity, rainfall and temperature give odd values, such as humidity being below 0% or above 100%

In [7]:
# Check if the range of values for some columns obey logical constraints
print(((data['Humidity (%)']<0.0) | (data['Humidity (%)']>100.0)).any())
print((data['Rainfall (mm)']<0.0).any())

False
False


And we can do the same for the other attributes like wind speed and cloud cover, but in this project, only important features (humidity, rainfall and temperatures) will be mainly used. Should there be a case of using the other attributes, then there will be checking for invalid values.

### Feature Engineering

What is feature engineering?

Feature engineering transforms raw data in columns into meaningful input features that are used for deeper exploratory data analysis and machine learning. In other words, deriving new features from existing ones. Several methods of feature engineering include:
- Creating new features such as splitting a datetime object into year, month and day.
- Feature transformation: Scaling (normalization/standardization) or encoding specific columns into usable formats
- Data Aggregation such as summarizing a column of data

After analyzing the data,
- It is not needed to create new features from splitting column values up, even for the datetime object. If it has to be done later in the project, it will be mentioned.
- Scaling will be looked into later; it may be optional for EDA. No requirement for encoding any column.
- There is no requirement for data aggregation at this early stage.

### Filtering data

This section separates the data, each corresponding to a unique city + state pairing. This allows for independent EDA and machine learning on the weather of each area/region in future project portions.

In [8]:
# How many unique cities are there?
print(f"Number of unique cities: {len(data['City'].unique().tolist())}")
print(f"Number of unique states: {len(data['State'].unique().tolist())}")

Number of unique cities: 10
Number of unique states: 10


Suppose we find all unique city + state pairs.

In [9]:
# Create a new column
new_column = data['City'] + ', ' + data['State']
data.insert(loc = 3, column = 'City_State', value = new_column)

In [12]:
unique_areas = data['City_State'].unique().tolist()
print(f"Number of unique city + state pairings: {len(unique_areas)}")
for i, item in enumerate(unique_areas, start=1):
    print(f"{i}: {item}")

Number of unique city + state pairings: 10
1: Mumbai, Maharashtra
2: Delhi, Delhi
3: Bengaluru, Karnataka
4: Chennai, Tamil Nadu
5: Kolkata, West Bengal
6: Hyderabad, Telangana
7: Ahmedabad, Gujarat
8: Jaipur, Rajasthan
9: Lucknow, Uttar Pradesh
10: Bhopal, Madhya Pradesh


In [14]:
# With the new column City_State, remove the city and state columns
data = data.drop(columns = ['City', 'State'])

In [15]:
subsets = []

# Separate the data into sets, one corresponding to each city + state
for item in data['City_State'].unique():
    subset = data[data['City_State']==item].copy()
    subsets.append(subset)

# Check (should be Mumbai, Maharashtra and the date column should be ordered logically)
subsets[0].head()

,Date,City_State,Temperature_Max (°C),Temperature_Min (°C),Temperature_Avg (°C),Humidity (%),Rainfall (mm),Wind_Speed (km/h),AQI,AQI_Category,Pressure (hPa),Cloud_Cover (%)
0,2024-01-01,"Mumbai, Maharashtra",32.5,18.0,25.2,77.6,0.0,3.3,259,Poor,1020.3,62.1
10,2024-01-02,"Mumbai, Maharashtra",25.6,14.3,19.9,50.4,0.0,11.4,63,Satisfactory,1016.4,26.7
20,2024-01-03,"Mumbai, Maharashtra",37.8,32.0,34.9,40.5,0.0,4.3,89,Satisfactory,1013.2,5.5
30,2024-01-04,"Mumbai, Maharashtra",26.0,15.7,20.9,65.1,60.8,13.9,123,Moderate,1001.3,80.5
40,2024-01-05,"Mumbai, Maharashtra",27.1,13.0,20.0,62.8,32.4,11.0,68,Satisfactory,990.4,91.0


For each dataset, check if the data is consistent (same number of records, date of each record ordered appropriately). The "Date" column is especially important here.

Number of records is 731 because there are 365 * 2 = 730 days in 2 years, and add an extra header row.

In [33]:
for i, subset in enumerate(subsets, start = 1):
    print(
        f"{i}: {subset['City_State'].iloc[0]}\n"
        f"Does each record have a unique date: {subset['Date'].is_unique}\n"
        f"Ordered date column: {subset['Date'].is_monotonic_increasing}\n"
        f"Number of records: {len(subset)}\n\n"
    )

1: Mumbai, Maharashtra
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


2: Delhi, Delhi
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


3: Bengaluru, Karnataka
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


4: Chennai, Tamil Nadu
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


5: Kolkata, West Bengal
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


6: Hyderabad, Telangana
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


7: Ahmedabad, Gujarat
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


8: Jaipur, Rajasthan
Does each record have a unique date: True
Ordered date column: True
Number of records: 731


9: Lucknow, Uttar Pradesh
Does each record have a unique date: True
Ordered dat

Each subset dataframe should have a single record for a single day's weather data for that dataframe's particular city and state it is in.

Label each dataframe the city and state, and export these datasets as csv files to be used in future project parts.

In [34]:
for subset in subsets:
    city_state = subset['City_State'].iloc[0]
    city_state = (city_state.replace(',', '_').replace(' ', ''))
    name = f"{city_state}.csv"
    subset.to_csv(name, index=False)